In [5]:
import pandas
import os

fn = '20200507_PEbert_QCselectCtrl_HGSVClibs.txt'

df = pandas.read_csv(fn, sep='\t', comment='#')
print(df.shape)
print(df.columns)

low_qual = df.loc[df['score'] == 0, :]
print(low_qual.shape)
lq_cells = set(low_qual['cell'].values)

controls = df.loc[df['score'] == -1, :]
print(controls.shape)
control_cells = set(controls['cell'].values)

high_qual = df.loc[df['score'] == 1, :]
print(high_qual.shape)
hq_cells = set(high_qual['cell'].values)

lq_control = lq_cells.intersection(control_cells)
if len(lq_control) != 0:
    raise ValueError('control in lq: {}'.format(sorted(lq_cells)))

hq_control = hq_cells.intersection(control_cells)
if len(hq_control) != 0:
    raise ValueError('control in hq: {}'.format(sorted(hq_control)))



blacklist = 'hgsvc_blacklist.txt'

blacklisted = control_cells.union(lq_cells)
print(len(blacklisted))

with open(blacklist, 'w') as dump:
    _ = dump.write('\n'.join(sorted(blacklisted)))


(1681, 5)
Index(['sample', 'cell', 'score', 'comments', 'reads'], dtype='object')
(425, 5)
(23, 5)
(1233, 5)
448
